In [37]:
# !pip install langchain langchain-experimental langchain-openai
# 
# !pip install azure-keyvault-secrets
# !pip install azure-identity
# !pip install yfinance
# !pip install langchain-tavily
# !pip install langfuse

In [38]:
from langchain_openai import AzureChatOpenAI
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_tavily import TavilySearch
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import json, os
from langfuse.callback import CallbackHandler


In [39]:
# KVUri = "https://dipanjans-kv.vault.azure.net"
# credential = DefaultAzureCredential()
# client = SecretClient(vault_url=KVUri, credential=credential)

# # #### Setting up secrets for future use ####
# # secretName_1 = "azure-openai-endpoint"
# # secretValue_1 = 'https://dipanjan-oai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview'

# # secretName_2 = "azure-openai-key"
# # secretValue_2 = 'E2EyDw7HealrkGoWMO3sbu9nCZ22Install pip dependenciesrN8oWo3MTVzePBnPczmfihbYJQQJ99BEACYeBjFXJ3w3AAABACOGr5SL'

# secretName_3 = "tavily-api-key"
# secretValue_3 = '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'

# secretName_4 = "langfuse-secret-key-1"
# secretValue_4 = "sk-lf-e32025cf-21ff-4e4e-82ea-751e13f47ee8"


# secretName_5 = "langfuse-public-key-1"
# secretValue_5 = "pk-lf-0acc3b60-04c7-4e3f-b2b6-8a0b5f9e3632"

# secretName_6 = "langfuse-host-1"
# secretValue_6 = "http://localhost:3000"

# client.set_secret(secretName_1, secretValue_1)
# client.set_secret(secretName_2, secretValue_2)
# client.set_secret(secretName_3, secretValue_3)
# client.set_secret(secretName_4, secretValue_4)
# client.set_secret(secretName_5, secretValue_5)
# client.set_secret(secretName_6, secretValue_6)

In [40]:
KVUri = "https://dipanjans-kv.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

oai_endpoint= client.get_secret("azure-openai-endpoint")
oai_key=      client.get_secret("azure-openai-key")
tavily_api_key =client.get_secret("tavily-api-key")
langfuse_sk= client.get_secret("langfuse-secret-key-1")
langfuse_pk= client.get_secret("langfuse-public-key-1")
langfuse_host= client.get_secret("langfuse-host-1")

os.environ['TAVILY_API_KEY'] = tavily_api_key.value


In [41]:
model = AzureChatOpenAI(api_version="2024-12-01-preview",
                        model='gpt-40-mini' , 
                        azure_endpoint= oai_endpoint.value,
                        api_key = oai_key.value)

# prompt = "Tell me a joke about mouse"
# model.invoke(prompt)

In [42]:
# creating the json parsers
class company(BaseModel):
    company_name: str = Field(description="company name for which stock code is required")
    company_code: str = Field(description="ticker code / company code /company symbol of the company")

class company_details(BaseModel):
    company_name: str = Field(description="company name for which stock code is required")
    stock_code: str = Field(description="ticker code / stock code / company code /company symbol of the company")
    newsdesc:str = Field(description= "summary of all news related to the company")
    sentiment:str= Field(description= "news sentiment- can take only one out of these values- [Positive, Negative, Neutral]")
    people_names:list[str] = Field(description= "list of all people names in the news articles")
    place_names:list[str] = Field(description= "list of all place / region / location names in the news articles")
    other_companies_referred:list[str] = Field(description= "list of other companies in the news articles")
    related_industries:list[str] = Field(description= "list of related industries")
    market_implications:str = Field(description= "what market implications can be expected, looking into the news articles")
    confidence_score:int =  Field(description= "confidence score for the report generated")


json_parser_company_code = JsonOutputParser(pydantic_object= company)
json_parser_company_details = JsonOutputParser(pydantic_object= company_details)
    

In [43]:
tavily_search_tool = TavilySearch(
    max_results=20,
    topic="news",
)
def get_news_tool(company_dct):
    data = {"comp": company_dct['company_name'], "code": company_dct['company_code']}
    news_searches= tavily_search_tool.invoke({"query":"Get latest news for the company - {comp} with company code {code}".format(**data)})
    all_news= ''
    for i,result in enumerate(news_searches['results']):
        news_num = i+1
        all_news = all_news + "news_{}".format(news_num) +": " +  str(result['content']) + "\n"
        
    return {'all_news':all_news}

# company_dct = {'company_name': 'Nvidia', 'company_code': 'NVDA'}
# get_news_tool(company_dct)
# # chain = get_stock_prompt | model | json_parser | get_news_tool
# # chain.invoke("nvidea")

In [44]:
langfuse_handler = CallbackHandler(
    secret_key= langfuse_sk.value,
    public_key=langfuse_pk.value,
    host=langfuse_host.value,
    session_id = 'dag_chain_finance_uuid'
)

In [45]:
def get_company_report(company_name):
    """
    Takes company name as input and generates a report in json. 
    The json is also saved in the format -> sample_output_json_<company_name>.json 
    """
    get_stock_prompt = PromptTemplate(template= "Give the stock code for \n{format_instructions}\n{inp_company}\n Give None if not listed",
                                          input_variables=["inp_company"],
                                          partial_variables={"format_instructions": json_parser_company_code.get_format_instructions()})
    get_report_prompt = PromptTemplate(template= "Give a detailed report in the following format \n{format_instructions}\n using the news given below - \n {all_news}",
                                          input_variables=["all_news"],
                                          partial_variables={"format_instructions": json_parser_company_details.get_format_instructions()})
    chain = get_stock_prompt | model | json_parser_company_code | get_news_tool | get_report_prompt | model | json_parser_company_details
    data = chain.invoke(company_name, config={"callbacks": [langfuse_handler]})
    
    #### save json
    with open("sample_output_json_{}.json".format(company_name), "w") as file:
        json.dump(data, file, indent=4)
    
    return data



# chain = get_stock_prompt | model | json_parser
# chain.invoke('nvidea')   -->  {'company_name': 'NVIDIA Corporation', 'company_code': 'NVDA'}

In [46]:
# get_company_report('Microsoft')
# get_company_report('Tredence')
# get_company_report('Tiger analytics')
# get_company_report('Vedanta limited')
get_company_report('Bajaj financial services')


{'company_name': 'Bajaj Finserv Ltd',
 'stock_code': 'BOM:532978',
 'newsdesc': 'Bajaj Finserv has reported a 10% rise in total income and a 14% jump in Q4FY25 net profit, highlighting strategic shifts in its product offerings. The company is also facing a fine of ₹ 850 crore for tax evasion while exploring partnerships for future growth, including a strategic collaboration with Bharti Airtel in financial services.',
 'sentiment': 'Positive',
 'people_names': [],
 'place_names': ['India', 'New Delhi'],
 'other_companies_referred': ['Bajaj Finance', 'Bharti Airtel', 'Allianz SE'],
 'related_industries': ['Financial Services',
  'Insurance',
  'Consumer Finance',
  'Banking'],
 'market_implications': 'The positive performance in income and profit suggests robust growth for Bajaj Finserv, while the strategic partnership with Bharti Airtel aims to enhance market access and service delivery in financial products. However, the tax evasion penalty poses risks to its financial standing and rep